In [1]:
#Uncomment if needed
#Install Cantonese NLP module
#!pip install --upgrade pycantonese

In [2]:
#Import data processing module
import pandas as pd
#pd.set_option("display.max_rows", None)

#Import NLP module
import pycantonese
from pycantonese.word_segmentation import Segmenter
import nltk
from nltk.corpus import stopwords

#Import word processing module
import re
import string

In [3]:
################################################
##### Keep only products with >=50 reviews #####
################################################

In [4]:
#Load the review dataset
review = pd.read_csv('/Users/spy1106/Documents/Xcceleratee/Projects/05_Capstone_Ecom/01_Web Scraping/Data/03_Reviews per item/df_review(beverages).csv')
review

,ProductID,Username,Review_Date,Comments,Rating,recommendOrNot
0,H29064,GRACE Ch,2022-04-24,飲開好味,5.0,0.0
1,H29064,joyce ho,2022-04-22,good,4.0,0.0
2,H29064,Ka Ying Sung,2022-04-22,good,5.0,0.0
3,H29064,Jane Kevin Watt,2022-04-18,小小凹，冇影響，會再回購,5.0,1.0
4,H29064,Jane Kevin Watt,2022-04-18,小小凹，但沒有爆,5.0,1.0
...,...,...,...,...,...,...
133614,A28526,Vivian Chang,2018-09-07,"包裝內全是紅茶粉</span></div><div class=""review-images...",1.0,0.0
133615,A28526,Nam CHENG,2018-05-07,okay啦。,4.0,0.0
133616,A28526,Amy Ch,2018-03-07,好,5.0,0.0
133617,A28526,Nam CHENG,2017-07-31,超值！,4.0,0.0


In [5]:
#Select only products with >=50 reviews
review = review[review['ProductID'].isin(review.ProductID.value_counts().loc[lambda x : x>=50].index.to_list())]
review

,ProductID,Username,Review_Date,Comments,Rating,recommendOrNot
0,H29064,GRACE Ch,2022-04-24,飲開好味,5.0,0.0
1,H29064,joyce ho,2022-04-22,good,4.0,0.0
2,H29064,Ka Ying Sung,2022-04-22,good,5.0,0.0
3,H29064,Jane Kevin Watt,2022-04-18,小小凹，冇影響，會再回購,5.0,1.0
4,H29064,Jane Kevin Watt,2022-04-18,小小凹，但沒有爆,5.0,1.0
...,...,...,...,...,...,...
133614,A28526,Vivian Chang,2018-09-07,"包裝內全是紅茶粉</span></div><div class=""review-images...",1.0,0.0
133615,A28526,Nam CHENG,2018-05-07,okay啦。,4.0,0.0
133616,A28526,Amy Ch,2018-03-07,好,5.0,0.0
133617,A28526,Nam CHENG,2017-07-31,超值！,4.0,0.0


In [6]:
#Check no. of products within the category
review.ProductID.nunique()

693

In [7]:
review = review.drop_duplicates()
review

,ProductID,Username,Review_Date,Comments,Rating,recommendOrNot
0,H29064,GRACE Ch,2022-04-24,飲開好味,5.0,0.0
1,H29064,joyce ho,2022-04-22,good,4.0,0.0
2,H29064,Ka Ying Sung,2022-04-22,good,5.0,0.0
3,H29064,Jane Kevin Watt,2022-04-18,小小凹，冇影響，會再回購,5.0,1.0
4,H29064,Jane Kevin Watt,2022-04-18,小小凹，但沒有爆,5.0,1.0
...,...,...,...,...,...,...
133614,A28526,Vivian Chang,2018-09-07,"包裝內全是紅茶粉</span></div><div class=""review-images...",1.0,0.0
133615,A28526,Nam CHENG,2018-05-07,okay啦。,4.0,0.0
133616,A28526,Amy Ch,2018-03-07,好,5.0,0.0
133617,A28526,Nam CHENG,2017-07-31,超值！,4.0,0.0


In [8]:
#Save updated review dataset
review.to_csv('/Users/spy1106/Downloads/df_review(beverages)_at_least_50.csv',index=False)

In [9]:
###############
##### NLP #####
###############

In [10]:
#Define the text as the Comments column
comments = review.Comments
comments

0                                                      飲開好味
1                                                     good 
2                                                      good
3                                              小小凹，冇影響，會再回購
4                                                  小小凹，但沒有爆
                                ...                        
133614    包裝內全是紅茶粉</span></div><div class="review-images...
133615                                               okay啦。
133616                                                    好
133617                                                  超值！
133618    發現好多疑似茶葉碎在外層透明膠包裝與茶包的包裝盒之間。所以失望!</span></div><...
Name: Comments, Length: 132867, dtype: object

In [11]:
#Define English stopwords (nltk stopwords + custom stopwords)
custom_eng_stop_words = ['divclass','div','review','span','spanclass','hktvmall','class','br','<br','l','src','https','nan','lt','href','gt','che','nil','amp','un','bb','ot','hktv','mall','images','img']
eng_stop_words = stopwords.words('english') + custom_eng_stop_words
eng_stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [12]:
#Define Chinese stopwords
custom_chi_stop_words = ['此','為','的','評論','內容','已','用戶','非','公開','嚟','系','但','而且','啦','味','麵','設定','啦','個人','認','今次','其實','圖片','還','雖然','不過','商品','亦','可惜','但是','但','是']
chi_stop_words = pycantonese.stop_words(add=custom_chi_stop_words,remove=['好','唔','唔使','唔係','冇'])
chi_stop_words

{'一啲',
 '一定',
 '不如',
 '不過',
 '之後',
 '乜',
 '乜嘢',
 '亦',
 '人哋',
 '今次',
 '但',
 '但係',
 '但是',
 '你',
 '你哋',
 '佢',
 '佢哋',
 '係',
 '個',
 '個人',
 '內容',
 '公開',
 '其他',
 '其實',
 '再',
 '到',
 '即',
 '即係',
 '原來',
 '去',
 '又',
 '可以',
 '可惜',
 '可能',
 '同',
 '同埋',
 '吖',
 '呀',
 '呢',
 '味',
 '咁',
 '咗',
 '咩',
 '咪',
 '哦',
 '哩',
 '哩個',
 '哩啲',
 '哩度',
 '哩樣',
 '商品',
 '啊',
 '啦',
 '啲',
 '喎',
 '喺',
 '喺度',
 '嗯',
 '嗰',
 '嗰個',
 '嗰啲',
 '嗰度',
 '嘅',
 '嘢',
 '噉',
 '噉樣',
 '嚟',
 '因為',
 '圖片',
 '多',
 '太',
 '如果',
 '就',
 '已',
 '已經',
 '幾',
 '幾多',
 '得',
 '想',
 '應該',
 '成日',
 '我',
 '我哋',
 '或者',
 '所以',
 '是',
 '最',
 '會',
 '有',
 '有冇',
 '有啲',
 '未',
 '梗係',
 '此',
 '為',
 '然之後',
 '用戶',
 '由',
 '的',
 '真係',
 '睇',
 '知',
 '系',
 '而',
 '而且',
 '而家',
 '自己',
 '要',
 '覺得',
 '設定',
 '評論',
 '話',
 '認',
 '諗',
 '講',
 '譬如',
 '跟住',
 '返',
 '過',
 '還',
 '邊個',
 '都',
 '雖然',
 '非',
 '麵',
 '點',
 '點樣',
 '點解'}

In [13]:
#Define punctuations
eng_punc = string.punctuation
chi_punc = "！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏. ..."
punc = eng_punc + chi_punc
punc

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~！？｡。＂＃＄％＆＇（）＊＋，－／：；＜＝＞＠［＼］＾＿｀｛｜｝～｟｠｢｣､、〃》「」『』【】〔〕〖〗〘〙〚〛〜〝〞〟〰〾〿–—‘’‛“”„‟…‧﹏. ...'

In [14]:
#Function to identify English text
def find_eng_words(word):
    if re.findall('[a-zA-Z]+',word):
        return re.findall('[a-zA-Z]+',word)[0]

In [15]:
#Function to remove emoji
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [16]:
#Function to check if there's number in the row value
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)

In [17]:
#Specify a specific linked text when segment
segmenter = Segmenter(allow={"唔好食","唔喜歡","唔鍾意",'好味','好食','抵食','抵買','杯麵','杯面','多次','回購','唔洗','平過','唔算','唔太','唔使','貨品'})

In [18]:
#Seperate Eng and Chi text
eng_list = [] #list to store english text
chi_list = [] #list to store chinese text
for sent in comments: #loop over ever comments sentence
    sent = remove_emoji(str(sent)) #remove emoji
    sent = sent.partition('</span>')[0] #remove hktvmall reply
    for word in sent.split(): #split the comments by space
        eng_word = find_eng_words(word) #take only the english alphabets
        if type(eng_word) == str and eng_word.lower() not in eng_stop_words and len(eng_word)>1: #if it's an english word (not a single letter) and not under stopwords
            eng_list.append(eng_word.lower()) #store the word in the english list
        elif eng_word == None: #if it's = None i.e. it is a chinese character
            for chi_word in pycantonese.segment(word, cls=segmenter):
                if chi_word not in chi_stop_words and chi_word not in punc and not chi_word.isdigit():
                    chi_list.append(chi_word) #store the word in the chinese list
print(eng_list,chi_list) 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [19]:
#Save segmented comments of occurence >= 30 as dataframe for manual stopwords listing
eng_df = pd.DataFrame({'Text': pd.Series(eng_list)}).Text.value_counts().loc[lambda x : x>=30].to_frame().reset_index()
chi_df = pd.DataFrame({'Text': pd.Series(chi_list)}).Text.value_counts().loc[lambda x : x>=30].to_frame().reset_index()
segmented_comments = pd.concat([eng_df,chi_df],axis=0)
segmented_comments.rename(columns = {'index':'Text','Text':'Occurrence'}, inplace = True)
segmented_comments

,Text,Occurrence
0,good,21028
1,ok,6580
2,pack,1069
3,nice,858
4,gd,810
...,...,...
1228,殘,30
1229,水質,30
1230,凹痕,30
1231,其它,30


In [115]:
#Save as csv
segmented_comments.to_csv('/Users/spy1106/Downloads/df_review(rice)_segmented.csv',index=False)

In [20]:
#Merge both chinese and english stopwords tgt
stop = chi_stop_words
stop |= set(eng_stop_words)
#df = pd.read_csv('/Users/spy1106/Downloads/df_review(noodles)_segmented_listed.csv')
#stop.update(df[df.Keep == 0].Text.unique())
stop

{'<br',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'amp',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'bb',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'br',
 'but',
 'by',
 'can',
 'che',
 'class',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'div',
 'divclass',
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'gt',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'hktv',
 'hktvmall',
 'how',
 'href',
 'https',
 'i',
 'if',
 'images',
 'img',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'l',
 'll',
 'lt',
 'm',
 'ma',
 'mall',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'nan',
 'needn',

In [21]:
'[' + '|'.join(punc) + ']'

'[!|"|#|$|%|&|\'|(|)|*|+|,|-|.|/|:|;|<|=|>|?|@|[|\\|]|^|_|`|{|||}|~|！|？|｡|。|＂|＃|＄|％|＆|＇|（|）|＊|＋|，|－|／|：|；|＜|＝|＞|＠|［|＼|］|＾|＿|｀|｛|｜|｝|～|｟|｠|｢|｣|､|、|〃|》|「|」|『|』|【|】|〔|〕|〖|〗|〘|〙|〚|〛|〜|〝|〞|〟|〰|〾|〿|–|—|‘|’|‛|“|”|„|‟|…|‧|﹏|.| |.|.|.]'

In [22]:
#Steps to clean comments row

#remove emoji
review['Clean_Comments'] = review['Comments'].apply(lambda x: remove_emoji(str(x)))

#remove hktvmall reply
review['Clean_Comments'] = review['Clean_Comments'].apply(lambda x: x.partition('</span>')[0])

#split comments with punctuations
review = review.assign(Clean_Comments=review.Clean_Comments.str.findall(r'[\w\s]+')).explode('Clean_Comments').reset_index(drop=True)
#review = review.assign(Clean_Comments=review.Clean_Comments.str.findall(r'\w+|[^\w\s]+')).explode('Clean_Comments').reset_index(drop=True)

#remove rows of punctuations
review['Clean_Comments'] = review['Clean_Comments'].str.replace('[^\w\s]','')

#remove english stopwords
review['Clean_Comments'] = review['Clean_Comments'].apply(lambda x: ' '.join([word for word in nltk.word_tokenize(str(x)) if word.lower() not in stop]))

#remove chinese stopwords
review['Clean_Comments'] = review['Clean_Comments'].apply(lambda x: ''.join([word for word in pycantonese.segment(x, cls=segmenter) if word not in stop]) if find_eng_words(x) == None else x)

#remove rows with unuseful comments
review = review[review.Clean_Comments != ""]

#remove rows with digits
review = review[review.Clean_Comments.apply(lambda x:has_numbers(x) == False)]

review

/Users/spy1106/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: The default value of regex will change from True to False in a future version.
  


,ProductID,Username,Review_Date,Comments,Rating,recommendOrNot,Clean_Comments
0,H29064,GRACE Ch,2022-04-24,飲開好味,5.0,0.0,飲開好味
1,H29064,joyce ho,2022-04-22,good,4.0,0.0,good
2,H29064,Ka Ying Sung,2022-04-22,good,5.0,0.0,good
3,H29064,Jane Kevin Watt,2022-04-18,小小凹，冇影響，會再回購,5.0,1.0,小小凹
4,H29064,Jane Kevin Watt,2022-04-18,小小凹，冇影響，會再回購,5.0,1.0,冇影響
...,...,...,...,...,...,...,...
217697,A28526,Nam CHENG,2018-05-07,okay啦。,4.0,0.0,okay啦
217698,A28526,Amy Ch,2018-03-07,好,5.0,0.0,好
217699,A28526,Nam CHENG,2017-07-31,超值！,4.0,0.0,超值
217700,A28526,Mabel Law,2017-05-24,發現好多疑似茶葉碎在外層透明膠包裝與茶包的包裝盒之間。所以失望!</span></div><...,1.0,0.0,發現好多疑似茶葉碎在外層透明膠包裝與茶包包裝盒之間


In [23]:
review = review.drop_duplicates()
review

,ProductID,Username,Review_Date,Comments,Rating,recommendOrNot,Clean_Comments
0,H29064,GRACE Ch,2022-04-24,飲開好味,5.0,0.0,飲開好味
1,H29064,joyce ho,2022-04-22,good,4.0,0.0,good
2,H29064,Ka Ying Sung,2022-04-22,good,5.0,0.0,good
3,H29064,Jane Kevin Watt,2022-04-18,小小凹，冇影響，會再回購,5.0,1.0,小小凹
4,H29064,Jane Kevin Watt,2022-04-18,小小凹，冇影響，會再回購,5.0,1.0,冇影響
...,...,...,...,...,...,...,...
217697,A28526,Nam CHENG,2018-05-07,okay啦。,4.0,0.0,okay啦
217698,A28526,Amy Ch,2018-03-07,好,5.0,0.0,好
217699,A28526,Nam CHENG,2017-07-31,超值！,4.0,0.0,超值
217700,A28526,Mabel Law,2017-05-24,發現好多疑似茶葉碎在外層透明膠包裝與茶包的包裝盒之間。所以失望!</span></div><...,1.0,0.0,發現好多疑似茶葉碎在外層透明膠包裝與茶包包裝盒之間


In [24]:
#Save as csv for comments classification
review.to_csv('/Users/spy1106/Downloads/df_review(beverages)_cleaned_2.csv')